In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import mysql.connector

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="e6a856a09ada43d789481ea5bd349625",
                                                           client_secret="7630a027718a45759d4c192004c7bf3f"))




def call_playlist(creator, playlist_id):
    cnx = mysql.connector.connect(user='root', password='0000',host='127.0.0.1', database='spotify')
    #step1

    playlist_features_list = ["artist","album","track_name", "track_id","popularity","danceability",
                              "energy","key","loudness","mode", "speechiness","instrumentalness",
                              "liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    #step2
    for i in range(0,1000,100):
        playlist = sp.user_playlist_tracks(creator, playlist_id, limit = 100, offset = i)["items"]
        
        for track in playlist:
            # Create empty dict
            playlist_features = {}
            
            # Get metadata
            playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
            playlist_features["album"] = track["track"]["album"]["name"]
            playlist_features["track_name"] = track["track"]["name"]
            playlist_features["track_id"] = track["track"]["id"]
            playlist_features["popularity"] = track["track"]["popularity"]
                    
            # Get audio features
            audio_features = sp.audio_features(playlist_features["track_id"])[0]
            for feature in playlist_features_list[5:]:
                playlist_features[feature] = audio_features[feature]
            
            placeholders = ', '.join(['%s'] * len(playlist_features))                
            mysql_insert_query = "INSERT INTO tracks_all VALUES (%s)" %placeholders
            cursor = cnx.cursor()
            cursor.execute(mysql_insert_query,list(playlist_features.values()))
            cnx.commit()

        
            # Concat the dfs
            track_df = pd.DataFrame(playlist_features, index = [0])
            playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

        #Step 3
        
    return playlist_df
cnx.close()